In [1]:
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import random
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
#from arctic import Arctic
#from fs import open_fs
import os
from itertools import combinations
import json
import requests
import datetime
import time
from collections import deque
import csv
import ccxt
from pathlib import Path
import calendar

In [2]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv

def write_to_csv(filename, exchange, data):
    p = Path("./data/", str(exchange))      #csv stored in 'data' folder of this notebook's directory address.
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)
        #print(data)
        
def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)
   # print("Im Done with {0}".format(len(olhv)))


In [3]:
#jupyter notebook --generate-config
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#The notebook server will temporarily stop sending output to the client in order to avoid crashing it. 
#Applied on the notebook as a command prompt on Anaconda compiler.


In [4]:
pwd   #Data will be stored inside the data folder of this notebook's directory.

'C:\\Users\\91626\\Work\\project\\Quant Res\\Cluster'

In [9]:
symbols = ['BTC','UNI','SUSHI','CRV','SOL','MANA','COMP','DOGE','BAT','MATIC','ENJ','XMR']

for symbol in symbols:
    scrape_candles_to_csv(f'{symbol}_USDT_5m.csv', 'binance', 3, f'{symbol}/USDT', '5m', '2021-05-1500:00:00Z', 1000)

1000 BTC/USDT candles in total from 2022-12-07T20:50:00.000Z to 2022-12-11T08:05:00.000Z
2000 BTC/USDT candles in total from 2022-12-04T09:30:00.000Z to 2022-12-11T08:05:00.000Z
3000 BTC/USDT candles in total from 2022-11-30T22:10:00.000Z to 2022-12-11T08:05:00.000Z
4000 BTC/USDT candles in total from 2022-11-27T10:50:00.000Z to 2022-12-11T08:05:00.000Z
5000 BTC/USDT candles in total from 2022-11-23T23:30:00.000Z to 2022-12-11T08:05:00.000Z
6000 BTC/USDT candles in total from 2022-11-20T12:10:00.000Z to 2022-12-11T08:05:00.000Z
7000 BTC/USDT candles in total from 2022-11-17T00:50:00.000Z to 2022-12-11T08:05:00.000Z
8000 BTC/USDT candles in total from 2022-11-13T13:30:00.000Z to 2022-12-11T08:05:00.000Z
9000 BTC/USDT candles in total from 2022-11-10T02:10:00.000Z to 2022-12-11T08:05:00.000Z
10000 BTC/USDT candles in total from 2022-11-06T14:50:00.000Z to 2022-12-11T08:05:00.000Z
11000 BTC/USDT candles in total from 2022-11-03T03:30:00.000Z to 2022-12-11T08:05:00.000Z
12000 BTC/USDT cand

93000 BTC/USDT candles in total from 2022-01-22T10:10:00.000Z to 2022-12-11T08:05:00.000Z
94000 BTC/USDT candles in total from 2022-01-18T22:50:00.000Z to 2022-12-11T08:05:00.000Z
95000 BTC/USDT candles in total from 2022-01-15T11:30:00.000Z to 2022-12-11T08:05:00.000Z
96000 BTC/USDT candles in total from 2022-01-12T00:10:00.000Z to 2022-12-11T08:05:00.000Z
97000 BTC/USDT candles in total from 2022-01-08T12:50:00.000Z to 2022-12-11T08:05:00.000Z
98000 BTC/USDT candles in total from 2022-01-05T01:30:00.000Z to 2022-12-11T08:05:00.000Z
99000 BTC/USDT candles in total from 2022-01-01T14:10:00.000Z to 2022-12-11T08:05:00.000Z
100000 BTC/USDT candles in total from 2021-12-29T02:50:00.000Z to 2022-12-11T08:05:00.000Z
101000 BTC/USDT candles in total from 2021-12-25T15:30:00.000Z to 2022-12-11T08:05:00.000Z
102000 BTC/USDT candles in total from 2021-12-22T04:10:00.000Z to 2022-12-11T08:05:00.000Z
103000 BTC/USDT candles in total from 2021-12-18T16:50:00.000Z to 2022-12-11T08:05:00.000Z
104000

17000 UNI/USDT candles in total from 2022-10-13T07:30:00.000Z to 2022-12-11T08:05:00.000Z
18000 UNI/USDT candles in total from 2022-10-09T20:10:00.000Z to 2022-12-11T08:05:00.000Z
19000 UNI/USDT candles in total from 2022-10-06T08:50:00.000Z to 2022-12-11T08:05:00.000Z
20000 UNI/USDT candles in total from 2022-10-02T21:30:00.000Z to 2022-12-11T08:05:00.000Z
21000 UNI/USDT candles in total from 2022-09-29T10:10:00.000Z to 2022-12-11T08:05:00.000Z
22000 UNI/USDT candles in total from 2022-09-25T22:50:00.000Z to 2022-12-11T08:05:00.000Z
23000 UNI/USDT candles in total from 2022-09-22T11:30:00.000Z to 2022-12-11T08:05:00.000Z
24000 UNI/USDT candles in total from 2022-09-19T00:10:00.000Z to 2022-12-11T08:05:00.000Z
25000 UNI/USDT candles in total from 2022-09-15T12:50:00.000Z to 2022-12-11T08:05:00.000Z
26000 UNI/USDT candles in total from 2022-09-12T01:30:00.000Z to 2022-12-11T08:05:00.000Z
27000 UNI/USDT candles in total from 2022-09-08T14:10:00.000Z to 2022-12-11T08:05:00.000Z
28000 UNI/

108000 UNI/USDT candles in total from 2021-12-01T08:10:00.000Z to 2022-12-11T08:05:00.000Z
109000 UNI/USDT candles in total from 2021-11-27T20:50:00.000Z to 2022-12-11T08:05:00.000Z
110000 UNI/USDT candles in total from 2021-11-24T09:30:00.000Z to 2022-12-11T08:05:00.000Z
111000 UNI/USDT candles in total from 2021-11-20T22:10:00.000Z to 2022-12-11T08:05:00.000Z
112000 UNI/USDT candles in total from 2021-11-17T10:50:00.000Z to 2022-12-11T08:05:00.000Z
113000 UNI/USDT candles in total from 2021-11-13T23:30:00.000Z to 2022-12-11T08:05:00.000Z
114000 UNI/USDT candles in total from 2021-11-10T12:10:00.000Z to 2022-12-11T08:05:00.000Z
115000 UNI/USDT candles in total from 2021-11-07T00:50:00.000Z to 2022-12-11T08:05:00.000Z
116000 UNI/USDT candles in total from 2021-11-03T13:30:00.000Z to 2022-12-11T08:05:00.000Z
117000 UNI/USDT candles in total from 2021-10-31T02:10:00.000Z to 2022-12-11T08:05:00.000Z
118000 UNI/USDT candles in total from 2021-10-27T14:50:00.000Z to 2022-12-11T08:05:00.000Z

31000 SUSHI/USDT candles in total from 2022-08-25T16:55:00.000Z to 2022-12-11T08:10:00.000Z
32000 SUSHI/USDT candles in total from 2022-08-22T05:35:00.000Z to 2022-12-11T08:10:00.000Z
33000 SUSHI/USDT candles in total from 2022-08-18T18:15:00.000Z to 2022-12-11T08:10:00.000Z
34000 SUSHI/USDT candles in total from 2022-08-15T06:55:00.000Z to 2022-12-11T08:10:00.000Z
35000 SUSHI/USDT candles in total from 2022-08-11T19:35:00.000Z to 2022-12-11T08:10:00.000Z
36000 SUSHI/USDT candles in total from 2022-08-08T08:15:00.000Z to 2022-12-11T08:10:00.000Z
37000 SUSHI/USDT candles in total from 2022-08-04T20:55:00.000Z to 2022-12-11T08:10:00.000Z
38000 SUSHI/USDT candles in total from 2022-08-01T09:35:00.000Z to 2022-12-11T08:10:00.000Z
39000 SUSHI/USDT candles in total from 2022-07-28T22:15:00.000Z to 2022-12-11T08:10:00.000Z
40000 SUSHI/USDT candles in total from 2022-07-25T10:55:00.000Z to 2022-12-11T08:10:00.000Z
41000 SUSHI/USDT candles in total from 2022-07-21T23:35:00.000Z to 2022-12-11T08

...

130000 MATIC/USDT candles in total from 2021-09-15T23:00:00.000Z to 2022-12-11T08:15:00.000Z
131000 MATIC/USDT candles in total from 2021-09-12T11:40:00.000Z to 2022-12-11T08:15:00.000Z
132000 MATIC/USDT candles in total from 2021-09-09T00:20:00.000Z to 2022-12-11T08:15:00.000Z
133000 MATIC/USDT candles in total from 2021-09-05T13:00:00.000Z to 2022-12-11T08:15:00.000Z
134000 MATIC/USDT candles in total from 2021-09-02T01:40:00.000Z to 2022-12-11T08:15:00.000Z
135000 MATIC/USDT candles in total from 2021-08-29T14:20:00.000Z to 2022-12-11T08:15:00.000Z
136000 MATIC/USDT candles in total from 2021-08-26T03:00:00.000Z to 2022-12-11T08:15:00.000Z
137000 MATIC/USDT candles in total from 2021-08-22T15:40:00.000Z to 2022-12-11T08:15:00.000Z
138000 MATIC/USDT candles in total from 2021-08-19T04:20:00.000Z to 2022-12-11T08:15:00.000Z
139000 MATIC/USDT candles in total from 2021-08-15T17:00:00.000Z to 2022-12-11T08:15:00.000Z
140000 MATIC/USDT candles in total from 2021-08-12T05:40:00.000Z to 20

53000 ENJ/USDT candles in total from 2022-06-10T07:40:00.000Z to 2022-12-11T08:15:00.000Z
54000 ENJ/USDT candles in total from 2022-06-06T20:20:00.000Z to 2022-12-11T08:15:00.000Z
55000 ENJ/USDT candles in total from 2022-06-03T09:00:00.000Z to 2022-12-11T08:15:00.000Z
56000 ENJ/USDT candles in total from 2022-05-30T21:40:00.000Z to 2022-12-11T08:15:00.000Z
57000 ENJ/USDT candles in total from 2022-05-27T10:20:00.000Z to 2022-12-11T08:15:00.000Z
58000 ENJ/USDT candles in total from 2022-05-23T23:00:00.000Z to 2022-12-11T08:15:00.000Z
59000 ENJ/USDT candles in total from 2022-05-20T11:40:00.000Z to 2022-12-11T08:15:00.000Z
60000 ENJ/USDT candles in total from 2022-05-17T00:20:00.000Z to 2022-12-11T08:15:00.000Z
61000 ENJ/USDT candles in total from 2022-05-13T13:00:00.000Z to 2022-12-11T08:15:00.000Z
62000 ENJ/USDT candles in total from 2022-05-10T01:40:00.000Z to 2022-12-11T08:15:00.000Z
63000 ENJ/USDT candles in total from 2022-05-06T14:20:00.000Z to 2022-12-11T08:15:00.000Z
64000 ENJ/

144000 ENJ/USDT candles in total from 2021-07-29T08:20:00.000Z to 2022-12-11T08:15:00.000Z
145000 ENJ/USDT candles in total from 2021-07-25T21:00:00.000Z to 2022-12-11T08:15:00.000Z
146000 ENJ/USDT candles in total from 2021-07-22T09:40:00.000Z to 2022-12-11T08:15:00.000Z
147000 ENJ/USDT candles in total from 2021-07-18T22:20:00.000Z to 2022-12-11T08:15:00.000Z
148000 ENJ/USDT candles in total from 2021-07-15T11:00:00.000Z to 2022-12-11T08:15:00.000Z
149000 ENJ/USDT candles in total from 2021-07-11T23:40:00.000Z to 2022-12-11T08:15:00.000Z
150000 ENJ/USDT candles in total from 2021-07-08T12:20:00.000Z to 2022-12-11T08:15:00.000Z
151000 ENJ/USDT candles in total from 2021-07-05T01:00:00.000Z to 2022-12-11T08:15:00.000Z
152000 ENJ/USDT candles in total from 2021-07-01T13:40:00.000Z to 2022-12-11T08:15:00.000Z
153000 ENJ/USDT candles in total from 2021-06-28T02:20:00.000Z to 2022-12-11T08:15:00.000Z
154000 ENJ/USDT candles in total from 2021-06-24T15:00:00.000Z to 2022-12-11T08:15:00.000Z

68000 XMR/USDT candles in total from 2022-04-19T05:40:00.000Z to 2022-12-11T08:15:00.000Z
69000 XMR/USDT candles in total from 2022-04-15T18:20:00.000Z to 2022-12-11T08:15:00.000Z
70000 XMR/USDT candles in total from 2022-04-12T07:00:00.000Z to 2022-12-11T08:15:00.000Z
71000 XMR/USDT candles in total from 2022-04-08T19:40:00.000Z to 2022-12-11T08:15:00.000Z
72000 XMR/USDT candles in total from 2022-04-05T08:20:00.000Z to 2022-12-11T08:15:00.000Z
73000 XMR/USDT candles in total from 2022-04-01T21:00:00.000Z to 2022-12-11T08:15:00.000Z
74000 XMR/USDT candles in total from 2022-03-29T09:40:00.000Z to 2022-12-11T08:15:00.000Z
75000 XMR/USDT candles in total from 2022-03-25T22:20:00.000Z to 2022-12-11T08:15:00.000Z
76000 XMR/USDT candles in total from 2022-03-22T11:00:00.000Z to 2022-12-11T08:15:00.000Z
77000 XMR/USDT candles in total from 2022-03-18T23:40:00.000Z to 2022-12-11T08:15:00.000Z
78000 XMR/USDT candles in total from 2022-03-15T12:20:00.000Z to 2022-12-11T08:15:00.000Z
79000 XMR/

160000 XMR/USDT candles in total from 2021-06-03T19:00:00.000Z to 2022-12-11T08:15:00.000Z
161000 XMR/USDT candles in total from 2021-05-31T07:40:00.000Z to 2022-12-11T08:15:00.000Z
162000 XMR/USDT candles in total from 2021-05-27T20:20:00.000Z to 2022-12-11T08:15:00.000Z
163000 XMR/USDT candles in total from 2021-05-24T09:00:00.000Z to 2022-12-11T08:15:00.000Z
164000 XMR/USDT candles in total from 2021-05-20T21:40:00.000Z to 2022-12-11T08:15:00.000Z
165000 XMR/USDT candles in total from 2021-05-17T10:20:00.000Z to 2022-12-11T08:15:00.000Z
166000 XMR/USDT candles in total from 2021-05-13T23:00:00.000Z to 2022-12-11T08:15:00.000Z
Saved 166000 candles from 2021-05-13T23:00:00.000Z to 2022-12-11T08:15:00.000Z to XMR_USDT_5m.csv


In [39]:
#To avoid notebook crashing symbols 
#symbols = ['LINK','BCH','XLM','ADA','BNB','LTC','ETH','AAVE','DOT','LINK','RUNE','YFI']
symbols=['LINK','BCH','XLM','ADA','BNB','LTC','ETH']
for symbol in symbols:
    scrape_candles_to_csv(f'{symbol}_USDT_5m.csv', 'binance', 3, f'{symbol}/USDT', '5m', '2021-05-1500:00:00Z', 1000)

1000 LINK/USDT candles in total from 2022-12-07T21:15:00.000Z to 2022-12-11T08:30:00.000Z
2000 LINK/USDT candles in total from 2022-12-04T09:55:00.000Z to 2022-12-11T08:30:00.000Z
3000 LINK/USDT candles in total from 2022-11-30T22:35:00.000Z to 2022-12-11T08:30:00.000Z
4000 LINK/USDT candles in total from 2022-11-27T11:15:00.000Z to 2022-12-11T08:30:00.000Z
5000 LINK/USDT candles in total from 2022-11-23T23:55:00.000Z to 2022-12-11T08:30:00.000Z
6000 LINK/USDT candles in total from 2022-11-20T12:35:00.000Z to 2022-12-11T08:30:00.000Z
7000 LINK/USDT candles in total from 2022-11-17T01:15:00.000Z to 2022-12-11T08:30:00.000Z
8000 LINK/USDT candles in total from 2022-11-13T13:55:00.000Z to 2022-12-11T08:30:00.000Z
9000 LINK/USDT candles in total from 2022-11-10T02:35:00.000Z to 2022-12-11T08:30:00.000Z
10000 LINK/USDT candles in total from 2022-11-06T15:15:00.000Z to 2022-12-11T08:30:00.000Z
11000 LINK/USDT candles in total from 2022-11-03T03:55:00.000Z to 2022-12-11T08:30:00.000Z
12000 LI

92000 LINK/USDT candles in total from 2022-01-25T21:55:00.000Z to 2022-12-11T08:30:00.000Z
93000 LINK/USDT candles in total from 2022-01-22T10:35:00.000Z to 2022-12-11T08:30:00.000Z
94000 LINK/USDT candles in total from 2022-01-18T23:15:00.000Z to 2022-12-11T08:30:00.000Z
95000 LINK/USDT candles in total from 2022-01-15T11:55:00.000Z to 2022-12-11T08:30:00.000Z
96000 LINK/USDT candles in total from 2022-01-12T00:35:00.000Z to 2022-12-11T08:30:00.000Z
97000 LINK/USDT candles in total from 2022-01-08T13:15:00.000Z to 2022-12-11T08:30:00.000Z
98000 LINK/USDT candles in total from 2022-01-05T01:55:00.000Z to 2022-12-11T08:30:00.000Z
99000 LINK/USDT candles in total from 2022-01-01T14:35:00.000Z to 2022-12-11T08:30:00.000Z
100000 LINK/USDT candles in total from 2021-12-29T03:15:00.000Z to 2022-12-11T08:30:00.000Z
101000 LINK/USDT candles in total from 2021-12-25T15:55:00.000Z to 2022-12-11T08:30:00.000Z
102000 LINK/USDT candles in total from 2021-12-22T04:35:00.000Z to 2022-12-11T08:30:00.0

16000 BCH/USDT candles in total from 2022-10-16T19:15:00.000Z to 2022-12-11T08:30:00.000Z
17000 BCH/USDT candles in total from 2022-10-13T07:55:00.000Z to 2022-12-11T08:30:00.000Z
18000 BCH/USDT candles in total from 2022-10-09T20:35:00.000Z to 2022-12-11T08:30:00.000Z
19000 BCH/USDT candles in total from 2022-10-06T09:15:00.000Z to 2022-12-11T08:30:00.000Z
20000 BCH/USDT candles in total from 2022-10-02T21:55:00.000Z to 2022-12-11T08:30:00.000Z
21000 BCH/USDT candles in total from 2022-09-29T10:35:00.000Z to 2022-12-11T08:30:00.000Z
22000 BCH/USDT candles in total from 2022-09-25T23:15:00.000Z to 2022-12-11T08:30:00.000Z
23000 BCH/USDT candles in total from 2022-09-22T11:55:00.000Z to 2022-12-11T08:30:00.000Z
24000 BCH/USDT candles in total from 2022-09-19T00:35:00.000Z to 2022-12-11T08:30:00.000Z
25000 BCH/USDT candles in total from 2022-09-15T13:15:00.000Z to 2022-12-11T08:30:00.000Z
26000 BCH/USDT candles in total from 2022-09-12T01:55:00.000Z to 2022-12-11T08:30:00.000Z
27000 BCH/

107000 BCH/USDT candles in total from 2021-12-04T19:55:00.000Z to 2022-12-11T08:30:00.000Z
108000 BCH/USDT candles in total from 2021-12-01T08:35:00.000Z to 2022-12-11T08:30:00.000Z
109000 BCH/USDT candles in total from 2021-11-27T21:15:00.000Z to 2022-12-11T08:30:00.000Z
110000 BCH/USDT candles in total from 2021-11-24T09:55:00.000Z to 2022-12-11T08:30:00.000Z
111000 BCH/USDT candles in total from 2021-11-20T22:35:00.000Z to 2022-12-11T08:30:00.000Z
112000 BCH/USDT candles in total from 2021-11-17T11:15:00.000Z to 2022-12-11T08:30:00.000Z
113000 BCH/USDT candles in total from 2021-11-13T23:55:00.000Z to 2022-12-11T08:30:00.000Z
114000 BCH/USDT candles in total from 2021-11-10T12:35:00.000Z to 2022-12-11T08:30:00.000Z
115000 BCH/USDT candles in total from 2021-11-07T01:15:00.000Z to 2022-12-11T08:30:00.000Z
116000 BCH/USDT candles in total from 2021-11-03T13:55:00.000Z to 2022-12-11T08:30:00.000Z
117000 BCH/USDT candles in total from 2021-10-31T02:35:00.000Z to 2022-12-11T08:30:00.000Z

31000 XLM/USDT candles in total from 2022-08-25T17:20:00.000Z to 2022-12-11T08:35:00.000Z
32000 XLM/USDT candles in total from 2022-08-22T06:00:00.000Z to 2022-12-11T08:35:00.000Z
33000 XLM/USDT candles in total from 2022-08-18T18:40:00.000Z to 2022-12-11T08:35:00.000Z
34000 XLM/USDT candles in total from 2022-08-15T07:20:00.000Z to 2022-12-11T08:35:00.000Z
35000 XLM/USDT candles in total from 2022-08-11T20:00:00.000Z to 2022-12-11T08:35:00.000Z
36000 XLM/USDT candles in total from 2022-08-08T08:40:00.000Z to 2022-12-11T08:35:00.000Z
37000 XLM/USDT candles in total from 2022-08-04T21:20:00.000Z to 2022-12-11T08:35:00.000Z
38000 XLM/USDT candles in total from 2022-08-01T10:00:00.000Z to 2022-12-11T08:35:00.000Z
39000 XLM/USDT candles in total from 2022-07-28T22:40:00.000Z to 2022-12-11T08:35:00.000Z
40000 XLM/USDT candles in total from 2022-07-25T11:20:00.000Z to 2022-12-11T08:35:00.000Z
41000 XLM/USDT candles in total from 2022-07-22T00:00:00.000Z to 2022-12-11T08:35:00.000Z
42000 XLM/

...

63000 BNB/USDT candles in total from 2022-05-06T14:40:00.000Z to 2022-12-11T08:35:00.000Z
64000 BNB/USDT candles in total from 2022-05-03T03:20:00.000Z to 2022-12-11T08:35:00.000Z
65000 BNB/USDT candles in total from 2022-04-29T16:00:00.000Z to 2022-12-11T08:35:00.000Z
66000 BNB/USDT candles in total from 2022-04-26T04:40:00.000Z to 2022-12-11T08:35:00.000Z
67000 BNB/USDT candles in total from 2022-04-22T17:20:00.000Z to 2022-12-11T08:35:00.000Z
68000 BNB/USDT candles in total from 2022-04-19T06:00:00.000Z to 2022-12-11T08:35:00.000Z
69000 BNB/USDT candles in total from 2022-04-15T18:40:00.000Z to 2022-12-11T08:35:00.000Z
70000 BNB/USDT candles in total from 2022-04-12T07:20:00.000Z to 2022-12-11T08:35:00.000Z
71000 BNB/USDT candles in total from 2022-04-08T20:00:00.000Z to 2022-12-11T08:35:00.000Z
72000 BNB/USDT candles in total from 2022-04-05T08:40:00.000Z to 2022-12-11T08:35:00.000Z
73000 BNB/USDT candles in total from 2022-04-01T21:20:00.000Z to 2022-12-11T08:35:00.000Z
74000 BNB/

154000 BNB/USDT candles in total from 2021-06-24T15:20:00.000Z to 2022-12-11T08:35:00.000Z
155000 BNB/USDT candles in total from 2021-06-21T04:00:00.000Z to 2022-12-11T08:35:00.000Z
156000 BNB/USDT candles in total from 2021-06-17T16:40:00.000Z to 2022-12-11T08:35:00.000Z
157000 BNB/USDT candles in total from 2021-06-14T05:20:00.000Z to 2022-12-11T08:35:00.000Z
158000 BNB/USDT candles in total from 2021-06-10T18:00:00.000Z to 2022-12-11T08:35:00.000Z
159000 BNB/USDT candles in total from 2021-06-07T06:40:00.000Z to 2022-12-11T08:35:00.000Z
160000 BNB/USDT candles in total from 2021-06-03T19:20:00.000Z to 2022-12-11T08:35:00.000Z
161000 BNB/USDT candles in total from 2021-05-31T08:00:00.000Z to 2022-12-11T08:35:00.000Z
162000 BNB/USDT candles in total from 2021-05-27T20:40:00.000Z to 2022-12-11T08:35:00.000Z
163000 BNB/USDT candles in total from 2021-05-24T09:20:00.000Z to 2022-12-11T08:35:00.000Z
164000 BNB/USDT candles in total from 2021-05-20T22:00:00.000Z to 2022-12-11T08:35:00.000Z

78000 LTC/USDT candles in total from 2022-03-15T12:40:00.000Z to 2022-12-11T08:35:00.000Z
79000 LTC/USDT candles in total from 2022-03-12T01:20:00.000Z to 2022-12-11T08:35:00.000Z
80000 LTC/USDT candles in total from 2022-03-08T14:00:00.000Z to 2022-12-11T08:35:00.000Z
81000 LTC/USDT candles in total from 2022-03-05T02:40:00.000Z to 2022-12-11T08:35:00.000Z
82000 LTC/USDT candles in total from 2022-03-01T15:20:00.000Z to 2022-12-11T08:35:00.000Z
83000 LTC/USDT candles in total from 2022-02-26T04:00:00.000Z to 2022-12-11T08:35:00.000Z
84000 LTC/USDT candles in total from 2022-02-22T16:40:00.000Z to 2022-12-11T08:35:00.000Z
85000 LTC/USDT candles in total from 2022-02-19T05:20:00.000Z to 2022-12-11T08:35:00.000Z
86000 LTC/USDT candles in total from 2022-02-15T18:00:00.000Z to 2022-12-11T08:35:00.000Z
87000 LTC/USDT candles in total from 2022-02-12T06:40:00.000Z to 2022-12-11T08:35:00.000Z
88000 LTC/USDT candles in total from 2022-02-08T19:20:00.000Z to 2022-12-11T08:35:00.000Z
89000 LTC/

2000 ETH/USDT candles in total from 2022-12-04T10:00:00.000Z to 2022-12-11T08:35:00.000Z
3000 ETH/USDT candles in total from 2022-11-30T22:40:00.000Z to 2022-12-11T08:35:00.000Z
4000 ETH/USDT candles in total from 2022-11-27T11:20:00.000Z to 2022-12-11T08:35:00.000Z
5000 ETH/USDT candles in total from 2022-11-24T00:00:00.000Z to 2022-12-11T08:35:00.000Z
6000 ETH/USDT candles in total from 2022-11-20T12:40:00.000Z to 2022-12-11T08:35:00.000Z
7000 ETH/USDT candles in total from 2022-11-17T01:20:00.000Z to 2022-12-11T08:35:00.000Z
8000 ETH/USDT candles in total from 2022-11-13T14:00:00.000Z to 2022-12-11T08:35:00.000Z
9000 ETH/USDT candles in total from 2022-11-10T02:40:00.000Z to 2022-12-11T08:35:00.000Z
10000 ETH/USDT candles in total from 2022-11-06T15:20:00.000Z to 2022-12-11T08:35:00.000Z
11000 ETH/USDT candles in total from 2022-11-03T04:00:00.000Z to 2022-12-11T08:35:00.000Z
12000 ETH/USDT candles in total from 2022-10-30T16:40:00.000Z to 2022-12-11T08:35:00.000Z
13000 ETH/USDT can

94000 ETH/USDT candles in total from 2022-01-18T23:20:00.000Z to 2022-12-11T08:35:00.000Z
95000 ETH/USDT candles in total from 2022-01-15T12:00:00.000Z to 2022-12-11T08:35:00.000Z
96000 ETH/USDT candles in total from 2022-01-12T00:40:00.000Z to 2022-12-11T08:35:00.000Z
97000 ETH/USDT candles in total from 2022-01-08T13:20:00.000Z to 2022-12-11T08:35:00.000Z
98000 ETH/USDT candles in total from 2022-01-05T02:00:00.000Z to 2022-12-11T08:35:00.000Z
99000 ETH/USDT candles in total from 2022-01-01T14:40:00.000Z to 2022-12-11T08:35:00.000Z
100000 ETH/USDT candles in total from 2021-12-29T03:20:00.000Z to 2022-12-11T08:35:00.000Z
101000 ETH/USDT candles in total from 2021-12-25T16:00:00.000Z to 2022-12-11T08:35:00.000Z
102000 ETH/USDT candles in total from 2021-12-22T04:40:00.000Z to 2022-12-11T08:35:00.000Z
103000 ETH/USDT candles in total from 2021-12-18T17:20:00.000Z to 2022-12-11T08:35:00.000Z
104000 ETH/USDT candles in total from 2021-12-15T06:00:00.000Z to 2022-12-11T08:35:00.000Z
10500

In [40]:
#pwd
#Checking the symbols
df=pd.DataFrame()
symbols= ['BTC','UNI','SUSHI','CRV','SOL','MANA','COMP','DOGE','BAT','MATIC','ENJ','XMR','LINK','BCH','XLM','ADA','BNB','LTC','ETH']
for i in symbols:
    if(i==symbols[0]):
        df['Time']=pd.read_csv('C:\\Users\\91626\\Work\\project\\Quant Res\\Cluster\\data\\Binance\\'+i+'_USDT_5m.csv',index_col=False,names=['Time','Open','Close','Low','High'])['Time']
        
    df[i]=pd.read_csv('C:\\Users\\91626\\Work\\project\\Quant Res\\Cluster\\data\\Binance\\'+i+'_USDT_5m.csv',index_col=False,names=['Time','Open','Close','Low','High'])['Close']

C:\Users\91626\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [41]:
df['Time'][0]

1620946200000

In [42]:
import datetime
h=[]
for i in df['Time']:
    #print(i)
    h.append(datetime.datetime.fromtimestamp(i/1000).isoformat())
df['Time']=h

In [43]:
h=[]
for i in df['Time']:
    dt_string=str(i)
    format = "%Y-%m-%dT%H:%M:%S"
    h.append(datetime.datetime.strptime(dt_string, format))
df['Time']=h

In [44]:
df.set_index('Time',inplace=True)

In [45]:
df.head()

,BTC,UNI,SUSHI,CRV,SOL,MANA,COMP,DOGE,BAT,MATIC,ENJ,XMR,LINK,BCH,XLM,ADA,BNB,LTC,ETH
Time,,,,,,,,,,,,,,,,,,,
2021-05-14 04:20:00,49401.05,37.073,14.623,3.334,40.399,1.2297,758.78,0.48720,1.1648,1.03230,1.9973,395.54,42.420,1214.00,0.64000,1.9071,561.52,309.03,3670.82
2021-05-14 04:25:00,49203.90,36.900,14.448,3.270,40.121,1.2207,749.73,0.51888,1.1664,1.01425,1.9870,395.03,42.353,1215.14,0.64041,1.9050,559.75,307.90,3652.62
2021-05-14 04:30:00,49012.79,36.759,14.450,3.222,40.000,1.2102,745.78,0.52000,1.1644,1.03030,2.0000,396.72,42.232,1206.61,0.64204,1.9173,562.22,309.76,3665.00
2021-05-14 04:35:00,48796.41,36.432,14.543,3.262,40.282,1.2238,749.23,0.51100,1.1752,1.03910,2.0220,399.91,42.464,1210.00,0.64285,1.9238,566.75,313.50,3681.24
2021-05-14 04:40:00,49141.07,36.472,14.633,3.297,40.602,1.2308,755.27,0.50435,1.1762,1.03684,2.0208,399.59,42.676,1234.29,0.64496,1.9344,568.58,314.50,3704.50
